<a href="https://colab.research.google.com/github/smpark0520/ESAA/blob/main/5_2_%EC%84%B8%EC%85%98_%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C_%EC%97%B0%EC%8A%B5%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Step 1: 필요한 라이브러리 불러오기**

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

#### **Step 2: 데이터 생성 (사용자-아이템 행렬 만들기)**

In [2]:
data = {
    "사용자": ["A", "A", "B", "B", "C", "C", "D", "D"],
    "아이템": ["영화1", "영화2", "영화1", "영화3", "영화2", "영화3", "영화1", "영화3"],
    "평점": [5, 3, 4, 2, 4, 5, 3, 4]
}
df = pd.DataFrame(data)
print("데이터프레임:")
print(df)

데이터프레임:
  사용자  아이템  평점
0   A  영화1   5
1   A  영화2   3
2   B  영화1   4
3   B  영화3   2
4   C  영화2   4
5   C  영화3   5
6   D  영화1   3
7   D  영화3   4


#### **Step 3: 사용자-아이템 행렬로 변환**

In [3]:
pivot_table = df.pivot(index="사용자", columns="아이템", values="평점").fillna(0)
print("\n사용자-아이템 행렬:")
print(pivot_table)


사용자-아이템 행렬:
아이템  영화1  영화2  영화3
사용자               
A    5.0  3.0  0.0
B    4.0  0.0  2.0
C    0.0  4.0  5.0
D    3.0  0.0  4.0


#### **Step 4: 아이템 기반 협업 필터링 - 코사인 유사도 계산**

In [6]:
similarity_matrix = cosine_similarity(pivot_table.T)  # 아이템 기반이므로 Transpose 적용
similarity_df = pd.DataFrame(similarity_matrix, index=pivot_table.columns, columns=pivot_table.columns)  # 데이터프레임으로 변환, 유사도행렬 생성
print("\n아이템 간 유사도 행렬:")
print("\n아이템 간 유사도 행렬:")
print(similarity_df)


아이템 간 유사도 행렬:

아이템 간 유사도 행렬:
아이템       영화1       영화2       영화3
아이템                              
영화1  1.000000  0.424264  0.421637
영화2  0.424264  1.000000  0.596285
영화3  0.421637  0.596285  1.000000


#### **Step 5: 특정 아이템과 가장 유사한 상위 2개 아이템 찾기**

In [8]:
item_name = "영화1"  # 예제 아이템
most_similar_items = similarity_df[item_name].sort_values(ascending=False)[1:3]
print(f"\n'영화1'과 가장 유사한 아이템:")
print(most_similar_items)


'영화1'과 가장 유사한 아이템:
아이템
영화2    0.424264
영화3    0.421637
Name: 영화1, dtype: float64


#### **Step 6: 사용자 기반 협업 필터링 - 유사도 행렬 계산 (빈칸 채우기)**

In [9]:
user_similarity_matrix = cosine_similarity(pivot_table)  # 사용자 기반이므로 pivot_table 그대로 사용
user_similarity_df = pd.DataFrame(user_similarity_matrix, index=pivot_table.index, columns=pivot_table.index)  # 데이터프레임으로 변환, 유사도행렬 생성                             # 데이터프레임으로 변환, 유사도행렬 생성
print("\n사용자 간 유사도 행렬:")
print(user_similarity_df)


사용자 간 유사도 행렬:
사용자         A         B         C         D
사용자                                        
A    1.000000  0.766965  0.321403  0.514496
B    0.766965  1.000000  0.349215  0.894427
C    0.321403  0.349215  1.000000  0.624695
D    0.514496  0.894427  0.624695  1.000000


#### **Step 7: 특정 사용자와 가장 유사한 상위 2명의 사용자 찾기 (빈칸 채우기)**

In [10]:
user_name = "A"
most_similar_users = user_similarity_df[user_name].sort_values(ascending=False)[1:3]  # 자기 자신 제외하고 상위 2명
print(f"\n사용자 '{user_name}'과 가장 유사한 사용자:")
print(most_similar_users)


사용자 'A'과 가장 유사한 사용자:
사용자
B    0.766965
D    0.514496
Name: A, dtype: float64


#### **Step 8: 협업 필터링을 적용하여 추천 (빈칸 채우기)**

In [12]:
def recommend_items(user, user_similarity_df, pivot_table, top_n=2):
    """ 주어진 사용자와 유사한 사용자의 선호도를 기반으로 아이템 추천 """
    similar_users = user_similarity_df[user].sort_values(ascending=False)[1:top_n+1].index

    recommended_items = pivot_table.loc[similar_users].mean().sort_values(ascending=False)[1:top_n+1]
    return recommended_items

recommended_movies = recommend_items("A", user_similarity_df, pivot_table, top_n=2)
print("\n사용자 A에게 추천되는 아이템:")
print(recommended_movies)


사용자 A에게 추천되는 아이템:
아이템
영화3    3.0
영화2    0.0
dtype: float64
